## Product Cost of Customer Acquisition
---
#### Another Practical Data Science Primer

The marketing team for an eCommerce platform has asked you to help identify how much they should spend to acquire 1 new customer.

This eCommerce site charges 10% from their customer's sales as their fee.

You are given three tables:
1. Invoice Table: information on every transaction
2. Product Table: contains details about the individual products sold.
3. Customer Table: details about the customer and their location.

##### Questions:
---
1. What is the eCommerce company's customer acquisition cost (CAC)?

    1.1 CAC = (Sales and Marketing Expense) / (Number of New Customers)

2. What is average Life Time Value (LTV) of a customer?

    2.1 What is the LTV to CAC ratio?
    
    2.2 Can the company afford to spend more to acquire a new customer?

3. What is the return rate, and which product is returned the most?

    3.1 Return rate = (total items returned) / (total items sold)

4. If the company decides to extend its market to another country, what is the feasible choice, and why?

5. Which was the most successful quarter in acquiring new customers?

    5.1 Note that this depends on multiple factors.

6. Devise a recommendation system based on the purchase data:

    6.1 If a customer buys product A and B, what is the probability that the customer will buy product C?
    
    6.2 What are the most purchased items by people who purchased product D? Hint: consider collaborative filtering methods.

In [1]:
# import needed libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# import local files for analysis

df_customer = pd.read_csv('./Customer_info_table.csv')
df_prod = pd.read_csv('./Product_info_table.csv')
df_inv = pd.read_csv('./Invoice_info_table.csv')

In [3]:
# View Customer Info table & stats:

print(f'Customer Info:')
print(f'{df_customer.info()}, \n')
print(f'Size of Cust Info Table: {df_customer.shape}')
print(f'# of unique customers: {df_customer.CustomerID.nunique()}')
print(f'Are null values listed: {df_customer.CustomerID.isnull().any()}\n')
display(df_customer.head())

Customer Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4389 entries, 0 to 4388
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerID  4380 non-null   float64
 1   Country     4389 non-null   object 
dtypes: float64(1), object(1)
memory usage: 68.7+ KB
None, 

Size of Cust Info Table: (4389, 2)
# of unique customers: 4372
Are null values listed: True



,CustomerID,Country
0,16143.0,United Kingdom
1,13983.0,United Kingdom
2,15854.0,United Kingdom
3,17634.0,United Kingdom
4,12933.0,United Kingdom


In [4]:
# Take a look at the Product Table:

print(f'Product Info:')
print(f'{df_prod.info()}, \n')
print(f'Size of Product Table: {df_prod.shape}')
print(f'# of unique Products: \n{df_prod.nunique()}\n')
display(df_prod.head())

Product Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18053 entries, 0 to 18052
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StockCode    18053 non-null  object 
 1   Description  17093 non-null  object 
 2   UnitPrice    18053 non-null  float64
dtypes: float64(1), object(2)
memory usage: 423.2+ KB
None, 

Size of Product Table: (18053, 3)
# of unique Products: 
StockCode      4070
Description    4211
UnitPrice      1630
dtype: int64



,StockCode,Description,UnitPrice
0,22027,TEA PARTY BIRTHDAY CARD,0.42
1,90214C,"""LETTER """"C"""" BLING KEY RING""",0.85
2,84748,FOLK FELT HANGING MULTICOL GARLAND,2.51
3,47585A,PINK FAIRY CAKE CUSHION COVER,4.21
4,90018A,SILVER M.O.P ORBIT DROP EARRINGS,4.24


In [5]:
# Take a look at the Customer Invoice Table:
print(f'Invoice Data: {df_inv.info()}\n')
print(f'Size of Invoice Table: {df_inv.shape}')
print(f'# of unique customers: {df_inv.CustomerID.nunique()}\n')
display(df_inv.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536480 entries, 0 to 536479
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    536480 non-null  object 
 1   StockCode    536480 non-null  object 
 2   Quantity     536480 non-null  int64  
 3   InvoiceDate  536480 non-null  object 
 4   CustomerID   401549 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 20.5+ MB
Invoice Data: None

Size of Invoice Table: (536480, 5)
# of unique customers: 4372



,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID
0,536408,22706,25,12/1/2010 11:41,14307.0
1,536528,22634,1,12/1/2010 13:17,15525.0
2,536529,22164,6,12/1/2010 13:20,14237.0
3,536544,22111,2,12/1/2010 14:32,NaN
4,536544,21238,4,12/1/2010 14:32,NaN


In [6]:
# notice 2 values for each table are the same.

print(f'# of unique customers by invoice: {df_inv.CustomerID.nunique()}')
print(f'# of unique customers by customer database: {df_customer.CustomerID.nunique()}')

# of unique customers by invoice: 4372
# of unique customers by customer database: 4372


#### Question 1: Customer Aquisition Cost
---

In [7]:
# Customer Info Table and the Customer Invoice table:
# Share the same amount of unique entries
# Perform a merge on the Customer ID colummn
# reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

df_ci = pd.merge(df_customer, df_inv, how='left', left_on='CustomerID', right_on='CustomerID')
print(f'Size of df_ci: {df_ci.shape}')

# reference for duplicates: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html
print(f'Check for duplicated values: {df_ci.duplicated().value_counts()}\n')
display(df_ci.head())

# Size of the dataframe and # of non-unique values are equal
# So the merge went well: all entries are unique in some way.

Size of df_ci: (1616852, 6)
Check for duplicated values: False    1616852
dtype: int64



,CustomerID,Country,InvoiceNo,StockCode,Quantity,InvoiceDate
0,16143.0,United Kingdom,537211,22666,6,12/5/2010 15:18
1,16143.0,United Kingdom,564190,23240,6,8/23/2011 16:01
2,16143.0,United Kingdom,552711,21218,6,5/11/2011 8:32
3,16143.0,United Kingdom,552711,21174,12,5/11/2011 8:32
4,16143.0,United Kingdom,543538,22667,6,2/9/2011 13:57


In [8]:
# Arrange data frame from highest to lowest Quantity

df_ci.InvoiceDate.value_counts()

10/31/2011 14:41    10017
12/8/2011 9:28       6741
12/9/2011 10:03      6579
12/5/2011 17:24      6489
6/29/2011 15:58      6345
                    ...  
4/28/2011 18:27         1
7/28/2011 13:40         1
9/2/2011 12:41          1
10/6/2011 12:37         1
12/2/2011 16:32         1
Name: InvoiceDate, Length: 23260, dtype: int64

#### Pull all invoice dates from the dataframe into new variables

In [9]:
dates = df_ci.InvoiceDate

print(f'The Data Frame \'dates\' is type: {type(dates)}')
print(f'Each Index of \'dates\' is type: {type(dates[0])}') # Panda Series of string data

print(f'\n{dates}')
print(f'\n1st array index for \'dates\' = {dates[0]}')

The Data Frame 'dates' is type: <class 'pandas.core.series.Series'>
Each Index of 'dates' is type: <class 'str'>

0          12/5/2010 15:18
1          8/23/2011 16:01
2           5/11/2011 8:32
3           5/11/2011 8:32
4           2/9/2011 13:57
                ...       
1616847    11/6/2011 13:53
1616848    12/1/2011 10:38
1616849    11/6/2011 13:53
1616850    12/1/2011 10:38
1616851    12/1/2011 10:38
Name: InvoiceDate, Length: 1616852, dtype: object

1st array index for 'dates' = 12/5/2010 15:18


In [10]:
date_list = dates.to_list()
display(date_list[0:25]) # display 1st 25 dates

['12/5/2010 15:18',
 '8/23/2011 16:01',
 '5/11/2011 8:32',
 '5/11/2011 8:32',
 '2/9/2011 13:57',
 '2/9/2011 13:57',
 '5/11/2011 8:32',
 '5/11/2011 8:32',
 '2/9/2011 13:57',
 '5/11/2011 8:32',
 '2/9/2011 13:57',
 '8/23/2011 16:01',
 '12/6/2011 14:13',
 '12/5/2010 15:18',
 '12/20/2010 12:51',
 '8/23/2011 16:01',
 '12/5/2010 15:18',
 '12/20/2010 12:51',
 '2/9/2011 13:57',
 '12/5/2010 15:18',
 '12/14/2010 10:00',
 '2/9/2011 13:57',
 '2/9/2011 13:57',
 '2/9/2011 13:57',
 '12/20/2010 12:51']

#### Example of `strip()` Method (Not Used Here):

In [11]:
display(date_list[0].strip(' ')[:-5])
date1 = date_list[0]
print(len(date1))
date1_strip = date1.strip(' ')[:-5]
print(date1_strip)
print(len(date1_strip))

'12/5/2010 '

15
12/5/2010 
10


#### Create New List Of Dates Without Time Value:
 - use `split()` method
 - alternative: use `strip()` method (not ideal in this case)

In [12]:
date_new = []
for i in date_list:
    #d = i.strip(' ')[:-5] # strip method not preferred: string data is not all the same length
    d = i.split()[0] # split at the space character. we only want the 1st value [0]
    date_new.append(d)

display(date_new[0:25])

['12/5/2010',
 '8/23/2011',
 '5/11/2011',
 '5/11/2011',
 '2/9/2011',
 '2/9/2011',
 '5/11/2011',
 '5/11/2011',
 '2/9/2011',
 '5/11/2011',
 '2/9/2011',
 '8/23/2011',
 '12/6/2011',
 '12/5/2010',
 '12/20/2010',
 '8/23/2011',
 '12/5/2010',
 '12/20/2010',
 '2/9/2011',
 '12/5/2010',
 '12/14/2010',
 '2/9/2011',
 '2/9/2011',
 '2/9/2011',
 '12/20/2010']

#### Append Data Frame With New Dates Stripped of Time:

In [13]:
display(df_ci.head())

,CustomerID,Country,InvoiceNo,StockCode,Quantity,InvoiceDate
0,16143.0,United Kingdom,537211,22666,6,12/5/2010 15:18
1,16143.0,United Kingdom,564190,23240,6,8/23/2011 16:01
2,16143.0,United Kingdom,552711,21218,6,5/11/2011 8:32
3,16143.0,United Kingdom,552711,21174,12,5/11/2011 8:32
4,16143.0,United Kingdom,543538,22667,6,2/9/2011 13:57


In [14]:
df_ci['InvDate'] = date_new
display(df_ci.head())

,CustomerID,Country,InvoiceNo,StockCode,Quantity,InvoiceDate,InvDate
0,16143.0,United Kingdom,537211,22666,6,12/5/2010 15:18,12/5/2010
1,16143.0,United Kingdom,564190,23240,6,8/23/2011 16:01,8/23/2011
2,16143.0,United Kingdom,552711,21218,6,5/11/2011 8:32,5/11/2011
3,16143.0,United Kingdom,552711,21174,12,5/11/2011 8:32,5/11/2011
4,16143.0,United Kingdom,543538,22667,6,2/9/2011 13:57,2/9/2011


#### Drop The Unwanted 'Invoice Date' Column:

In [15]:
df_ci.drop(['InvoiceDate'], axis=1, inplace=True)
display(df_ci.head())

,CustomerID,Country,InvoiceNo,StockCode,Quantity,InvDate
0,16143.0,United Kingdom,537211,22666,6,12/5/2010
1,16143.0,United Kingdom,564190,23240,6,8/23/2011
2,16143.0,United Kingdom,552711,21218,6,5/11/2011
3,16143.0,United Kingdom,552711,21174,12,5/11/2011
4,16143.0,United Kingdom,543538,22667,6,2/9/2011


#### Create New Data Frame Based On Dates:

In [16]:
# original data frame shape:
print(df_ci.shape, '\n')

# create new DF with only 2011 dates:
new_df = df_ci[df_ci.InvDate >= '2011']
print(new_df.shape, '\n')
display(new_df.InvDate.value_counts())
display(new_df)

(1616852, 6) 

(758444, 6) 



8/30/2011    25861
4/18/2011    16925
9/21/2011    13826
7/5/2011     13199
5/10/2011    12429
             ...  
7/3/2011       581
8/14/2011      550
8/7/2011       541
3/13/2011      525
5/1/2011       450
Name: InvDate, Length: 177, dtype: int64

,CustomerID,Country,InvoiceNo,StockCode,Quantity,InvDate
1,16143.0,United Kingdom,564190,23240,6,8/23/2011
2,16143.0,United Kingdom,552711,21218,6,5/11/2011
3,16143.0,United Kingdom,552711,21174,12,5/11/2011
6,16143.0,United Kingdom,552711,21430,4,5/11/2011
7,16143.0,United Kingdom,552711,21535,6,5/11/2011
...,...,...,...,...,...,...
1616735,17051.0,United Kingdom,554993,22352,3,5/29/2011
1616736,17051.0,United Kingdom,554993,23204,2,5/29/2011
1616737,17051.0,United Kingdom,554993,22367,2,5/29/2011
1616738,17051.0,United Kingdom,554993,21213,12,5/29/2011


#### Split Data Frame Into Quarterly Transactions:

In [17]:
new_df['Quarter'] = pd.PeriodIndex(new_df.InvDate, freq='Q')

/tmp/ipykernel_719387/3776867125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Quarter'] = pd.PeriodIndex(new_df.InvDate, freq='Q')


In [18]:
print(new_df.Quarter.nunique())
display(new_df)

3


,CustomerID,Country,InvoiceNo,StockCode,Quantity,InvDate,Quarter
1,16143.0,United Kingdom,564190,23240,6,8/23/2011,2011Q3
2,16143.0,United Kingdom,552711,21218,6,5/11/2011,2011Q2
3,16143.0,United Kingdom,552711,21174,12,5/11/2011,2011Q2
6,16143.0,United Kingdom,552711,21430,4,5/11/2011,2011Q2
7,16143.0,United Kingdom,552711,21535,6,5/11/2011,2011Q2
...,...,...,...,...,...,...,...
1616735,17051.0,United Kingdom,554993,22352,3,5/29/2011,2011Q2
1616736,17051.0,United Kingdom,554993,23204,2,5/29/2011,2011Q2
1616737,17051.0,United Kingdom,554993,22367,2,5/29/2011,2011Q2
1616738,17051.0,United Kingdom,554993,21213,12,5/29/2011,2011Q2


#### Number Of Unique Customers By Quarter:

 - Note the growth of in each quarter.
 - Can be used to calculate new customers.

In [19]:
display(new_df.groupby('Quarter').agg({'CustomerID':'nunique'}))

,CustomerID
Quarter,
2011Q1,1020
2011Q2,2024
2011Q3,2196


#### Check The Products and Invoice DataFrames For Commonalities:

 - Note that each has a stock code and the invoices have dates

In [20]:
display(df_prod)

,StockCode,Description,UnitPrice
0,22027,TEA PARTY BIRTHDAY CARD,0.42
1,90214C,"""LETTER """"C"""" BLING KEY RING""",0.85
2,84748,FOLK FELT HANGING MULTICOL GARLAND,2.51
3,47585A,PINK FAIRY CAKE CUSHION COVER,4.21
4,90018A,SILVER M.O.P ORBIT DROP EARRINGS,4.24
...,...,...,...
18048,DOT,DOTCOM POSTAGE,204.85
18049,B,Adjust bad debt,-11062.06
18050,22778,GLASS CLOCHE SMALL,0.00
18051,21891,TRADITIONAL WOODEN SKIPPING ROPE,0.72


In [21]:
display(df_inv)

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID
0,536408,22706,25,12/1/2010 11:41,14307.0
1,536528,22634,1,12/1/2010 13:17,15525.0
2,536529,22164,6,12/1/2010 13:20,14237.0
3,536544,22111,2,12/1/2010 14:32,NaN
4,536544,21238,4,12/1/2010 14:32,NaN
...,...,...,...,...,...
536475,581451,22310,6,12/8/2011 17:57,17144.0
536476,581488,23118,16,12/9/2011 9:45,17428.0
536477,581492,22108,2,12/9/2011 10:03,NaN
536478,581497,20724,55,12/9/2011 10:23,NaN


#### Merge Both Data Frames:

In [22]:
df_pi = pd.merge(df_inv, df_prod, how='right', left_on='StockCode', right_on='StockCode')
display(df_pi)

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Description,UnitPrice
0,552893,22027,12,5/12/2011 10:49,14421.0,TEA PARTY BIRTHDAY CARD,0.42
1,579708,22027,12,11/30/2011 14:16,12682.0,TEA PARTY BIRTHDAY CARD,0.42
2,541213,22027,12,1/14/2011 13:28,15033.0,TEA PARTY BIRTHDAY CARD,0.42
3,543190,22027,12,2/4/2011 11:52,17226.0,TEA PARTY BIRTHDAY CARD,0.42
4,537434,22027,2,12/6/2010 16:57,NaN,TEA PARTY BIRTHDAY CARD,0.42
...,...,...,...,...,...,...,...
4061125,568716,22946,1,9/28/2011 16:13,NaN,check,0.00
4061126,572037,22946,4,10/20/2011 12:07,17365.0,check,0.00
4061127,C579502,22946,-1,11/29/2011 16:19,16759.0,check,0.00
4061128,572550,22946,1,10/24/2011 17:06,NaN,check,0.00


#### Add Column For Total Dollars Spent Per Invoice:

In [23]:
df_pi['total'] = df_pi.Quantity * df_pi.UnitPrice
display(df_pi.head())

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Description,UnitPrice,total
0,552893,22027,12,5/12/2011 10:49,14421.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
1,579708,22027,12,11/30/2011 14:16,12682.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
2,541213,22027,12,1/14/2011 13:28,15033.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
3,543190,22027,12,2/4/2011 11:52,17226.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
4,537434,22027,2,12/6/2010 16:57,NaN,TEA PARTY BIRTHDAY CARD,0.42,0.84


In [24]:
df_pi2 = df_pi[df_pi.InvoiceDate >= '2011']
display(df_pi2)

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Description,UnitPrice,total
0,552893,22027,12,5/12/2011 10:49,14421.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
5,554669,22027,12,5/25/2011 15:31,15203.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
7,566949,22027,1,9/15/2011 16:32,NaN,TEA PARTY BIRTHDAY CARD,0.42,0.42
8,545419,22027,12,3/2/2011 14:16,15104.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
9,546683,22027,12,3/16/2011 9:38,12868.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
...,...,...,...,...,...,...,...,...
4061117,563547,22946,6,8/17/2011 12:29,NaN,check,0.00,0.00
4061119,557966,22946,24,6/24/2011 9:38,15187.0,check,0.00,0.00
4061122,566485,22946,2,9/13/2011 9:49,13880.0,check,0.00,0.00
4061124,567184,22946,3,9/18/2011 15:41,16033.0,check,0.00,0.00


In [25]:
dates2 = df_pi2.InvoiceDate

print(f'The Data Frame \'dates\' is type: {type(dates2)}')
print(f'Each Index of \'dates\' is type: {type(dates2[0])}') # Panda Series of string data

print(f'\n{dates2}')
print(f'\n1st array index for \'dates\' = {dates2[0]}')

The Data Frame 'dates' is type: <class 'pandas.core.series.Series'>
Each Index of 'dates' is type: <class 'str'>

0          5/12/2011 10:49
5          5/25/2011 15:31
7          9/15/2011 16:32
8           3/2/2011 14:16
9           3/16/2011 9:38
                ...       
4061117    8/17/2011 12:29
4061119     6/24/2011 9:38
4061122     9/13/2011 9:49
4061124    9/18/2011 15:41
4061125    9/28/2011 16:13
Name: InvoiceDate, Length: 2079268, dtype: object

1st array index for 'dates' = 5/12/2011 10:49


In [26]:
date_list2 = dates2.to_list()
display(date_list2[0:25]) # display 1st 25 dates

['5/12/2011 10:49',
 '5/25/2011 15:31',
 '9/15/2011 16:32',
 '3/2/2011 14:16',
 '3/16/2011 9:38',
 '6/1/2011 17:18',
 '7/14/2011 15:30',
 '7/17/2011 14:59',
 '5/5/2011 15:42',
 '3/28/2011 11:36',
 '7/19/2011 10:42',
 '3/17/2011 18:15',
 '8/11/2011 15:25',
 '3/30/2011 16:34',
 '7/12/2011 14:17',
 '7/21/2011 19:17',
 '7/28/2011 15:36',
 '8/10/2011 16:19',
 '7/6/2011 15:49',
 '6/13/2011 15:30',
 '8/25/2011 19:39',
 '6/5/2011 15:06',
 '5/3/2011 12:02',
 '8/10/2011 16:51',
 '4/7/2011 13:07']

In [27]:
date_new2 = []
for i in date_list2:
    #d = i.strip(' ')[:-5] # strip method not preferred: string data is not all the same length
    d = i.split()[0] # split at the space character. we only want the 1st value [0]
    date_new2.append(d)

display(date_new2[0:25])

['5/12/2011',
 '5/25/2011',
 '9/15/2011',
 '3/2/2011',
 '3/16/2011',
 '6/1/2011',
 '7/14/2011',
 '7/17/2011',
 '5/5/2011',
 '3/28/2011',
 '7/19/2011',
 '3/17/2011',
 '8/11/2011',
 '3/30/2011',
 '7/12/2011',
 '7/21/2011',
 '7/28/2011',
 '8/10/2011',
 '7/6/2011',
 '6/13/2011',
 '8/25/2011',
 '6/5/2011',
 '5/3/2011',
 '8/10/2011',
 '4/7/2011']

In [28]:
display(df_pi2.head())

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Description,UnitPrice,total
0,552893,22027,12,5/12/2011 10:49,14421.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
5,554669,22027,12,5/25/2011 15:31,15203.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
7,566949,22027,1,9/15/2011 16:32,NaN,TEA PARTY BIRTHDAY CARD,0.42,0.42
8,545419,22027,12,3/2/2011 14:16,15104.0,TEA PARTY BIRTHDAY CARD,0.42,5.04
9,546683,22027,12,3/16/2011 9:38,12868.0,TEA PARTY BIRTHDAY CARD,0.42,5.04


#### Remove Time Data From 'InvoiceDate':

In [29]:
df_pi2['Date'] = date_new2
display(df_pi2.head())

/tmp/ipykernel_719387/919419652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pi2['Date'] = date_new2


,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Description,UnitPrice,total,Date
0,552893,22027,12,5/12/2011 10:49,14421.0,TEA PARTY BIRTHDAY CARD,0.42,5.04,5/12/2011
5,554669,22027,12,5/25/2011 15:31,15203.0,TEA PARTY BIRTHDAY CARD,0.42,5.04,5/25/2011
7,566949,22027,1,9/15/2011 16:32,NaN,TEA PARTY BIRTHDAY CARD,0.42,0.42,9/15/2011
8,545419,22027,12,3/2/2011 14:16,15104.0,TEA PARTY BIRTHDAY CARD,0.42,5.04,3/2/2011
9,546683,22027,12,3/16/2011 9:38,12868.0,TEA PARTY BIRTHDAY CARD,0.42,5.04,3/16/2011


#### Drop The Unwanted 'InvoiceDate' Column:

In [30]:
df_pi2.drop(['InvoiceDate'], axis=1, inplace=True)
display(df_pi2.head())

/tmp/ipykernel_719387/3345535776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pi2.drop(['InvoiceDate'], axis=1, inplace=True)


,InvoiceNo,StockCode,Quantity,CustomerID,Description,UnitPrice,total,Date
0,552893,22027,12,14421.0,TEA PARTY BIRTHDAY CARD,0.42,5.04,5/12/2011
5,554669,22027,12,15203.0,TEA PARTY BIRTHDAY CARD,0.42,5.04,5/25/2011
7,566949,22027,1,NaN,TEA PARTY BIRTHDAY CARD,0.42,0.42,9/15/2011
8,545419,22027,12,15104.0,TEA PARTY BIRTHDAY CARD,0.42,5.04,3/2/2011
9,546683,22027,12,12868.0,TEA PARTY BIRTHDAY CARD,0.42,5.04,3/16/2011


#### Group By Invoice Number and Stock Code:

In [39]:
dff = df_pi2.groupby(['InvoiceNo', 'StockCode']).agg({'total':['sum']})
display(dff)
dff.reset_index()

total
                         sum
InvoiceNo StockCode         
545220    17091J      809.64
          20829      1072.85
          21259       576.98
          21485       305.99
          21624      1091.68
...                      ...
C569128   21733       360.21
          22726       444.10
          22727       316.22
          22729       159.10
C569138   23118       367.53

[261895 rows x 1 columns]

,InvoiceNo,StockCode,total
,,,sum
0,545220,17091J,809.64
1,545220,20829,1072.85
2,545220,21259,576.98
3,545220,21485,305.99
4,545220,21624,1091.68
...,...,...,...
261890,C569128,21733,360.21
261891,C569128,22726,444.10
261892,C569128,22727,316.22


In [40]:
dff_neg = dff[dff[('total', 'sum')] < 0]
display(dff_neg)

total
                           sum
InvoiceNo StockCode           
545226    21115     -151813.39
          21441       -3104.83
          22279       -1939.52
          22669          -8.25
          22917          -2.53
...                        ...
C568532   22734      -87751.44
C568571   84625A       -196.64
C568575   72803A     -51359.21
C568799   M           -5532.76
C568899   23292       -1142.57

[15053 rows x 1 columns]

In [ ]:
# This can take 4-5 minutes
df_pi2['quarter'] = pd.PeriodIndex(df_pi2.InvoiceDate, freq='Q')

In [ ]:
print(df_pi2.shape)
df_pi2.groupby('quarter').agg({'CustomerID':'nunique', 'total':'sum'})

#### Calculate Return Rate:

In [35]:
display(df_pi2.total.value_counts())

 0.00         133732
 15.00         19112
 1.25          13873
 4.13          13488
 16.50         13058
               ...  
 189680.00         1
 231610.00         1
 47.30             1
 2023.68           1
-10226.70          1
Name: total, Length: 22013, dtype: int64

In [36]:
t = df_pi2.total.to_numpy()
print(t.min())
print(t.max())

-52609500.0
7131510.0


#### Sort And Find Negative Values:

In [37]:
t.sort()
t_neg = []
for i in t:
    if i < 0:
        t_neg.append(i)

print(t_neg)
print(len(t_neg))

[-52609500.0, -18705600.0, -9355500.0, -5788300.5, -5617431.000000001, -5611680.0, -5611680.0, -5331582.0, -4676400.0, -4260532.5, -4131810.0, -3974940.0, -3741120.0, -3741120.0, -3582832.4999999995, -3488076.0000000005, -3375000.0, -3326400.0, -3216942.0, -2860298.9999999995, -2771644.5, -2744685.0, -2587950.0, -2521611.0, -2470284.0, -2316397.5, -2277679.5, -2149861.5, -2058062.4000000001, -2006261.9999999998, -1997308.8000000003, -1938316.5, -1895673.6, -1870560.0, -1752840.0000000002, -1733130.0, -1691482.5, -1676673.0, -1641761.9999999998, -1552473.0, -1534005.0, -1526715.0000000002, -1520100.0, -1514856.0, -1513593.0, -1469088.0, -1403635.5, -1363950.0, -1344859.2, -1273896.0, -1255365.0, -1248196.5, -1240204.8, -1224450.0, -1200000.0, -1173892.5, -1156248.0, -1151280.0, -1148175.0, -1143801.6, -1109565.0, -1070982.0, -1039851.0, -1031562.0, -1016995.2, -997920.0, -997920.0, -985473.6000000001, -975888.0, -974250.0, -968260.5, -952357.5000000001, -948213.0, -920767.4999999999, -9